Dataset used:

[Munich Germany](https://mapzen.com/data/metro-extracts/metro/munich_germany/) from OpenStreetMap.  
[Download(OSMXML 39MB)](https://s3.amazonaws.com/metro-extracts.mapzen.com/munich_germany.osm.bz2)

In [ ]:
!curl https://s3.amazonaws.com/metro-extracts.mapzen.com/munich_germany.osm.bz2

将下载文件解压后命名为"munich_germany.osm"。解压后文件大小为481MB，为了简便测试，使用以下代码来生成小样本：

In [ ]:
import sampling
# 生成10分之一的测试数据。
K10 = sampling.writeSample("munich_germany", 10)

运行代码，生成大小约为50MB的样本数据。

In [ ]:
!du -sh munich_germany_k10.osm

# 审核数据 audit.py

在审核osm数据时第一个遇到的问题是德语的特殊字符,例如 ä, ü，ß 等不能被python很好的解析，所以在源代码中作以下处理：
1. 在第一或二行加入编码声明 [详情](https://www.python.org/peps/pep-0263.html)
```python
#coding:utf-8
```
2. 在打开文件时添加编码参数
```python
codecs.open(osmfile, mode='r', encoding='utf-8')
```
3. 设置python默认编码环境
```python
reload(sys)
sys.setdefaultencoding('utf8')
```

### 审核街道名称
在使用audit.py对样本进行审核后，发现由于德语的特殊性，仅简单的对街道名的词尾进行分类已不适用。例如，很多德语街道（strasse）名才取连写的方式：

Rosen<span style="color:blue">straße</span> （玫瑰街）


而非：
Rosen <span style="color:blue">straße</span> （玫瑰 街）

但同时也存在以下几种形式：

abc<span style="color:blue">straße</span>

abc-<span style="color:blue">straße</span>

abc <span style="color:blue">straße</span>

考虑到针对不同的语言有不同的分词的方法，而我们的重点是数据审核和清理，这里只采用以下正则表达式：

```
street_type_re = re.compile(
    ur'(\s|-)?(straße|weg|ring|platz|allee|bogen|gasse|brücke|hof|berg|eck)$',
    re.IGNORECASE | re.UNICODE) 
```
*(我花了较长时间来使得以上的re有match，主要是在python2中，不仅compile方法需要有额外的re.UNICODE flag，还必须将string const前标注ur来转译成"unicode string")*

在德语中除了straße（街）以外，还有weg（道），ring（环），platz（广场），allee（大道），gasse（通道），brücke（桥），hof（庄）等等。这些多次出现的词尾是经过了多次运行audit.py后得到的印象，这里将它们都加入到正则表达式的预期街道名中。

在除去了以上的街道名词尾后，结果中还包涵了以下一类以“介词”开头或结尾的街道名，大多数是在POI附近的街道名，表示“在”什么附近，“靠近”某某等等，比如：
```
Am Krautgarten (在。。。）
Im Wismat (靠。。。)
Zu Maria-Eich (至。。。)
Zur Allacher
```
*(在德语中介词需要根据之后的名词属性变格，比如“至。。。”（英文 to)可能有"zu", "zum", "zur"等多种形式)*

还有针对地貌或者属性相关的街道名，例如：
```
Pratalinsel (。。。岛）
Englishgarten (。。。花园）
Rindermarkt (。。。市场)
```

在观察这些街道名之后，我将他们都加入了通过审核的字典中。

# 数据清理
在多次调整audit.py的预期街道名之后，通过第三方地图服务（在maps.google.de）验证余下的可怀疑街道名，例如：
```
<node changeset="22627081" id="2888062656" lat="48.0928659" lon="11.5542587" timestamp="2014-05-29T20:37:14Z" uid="78613" user="heilbron" version="1">
                <tag k="addr:city" v="München" />
                <tag k="addr:street" v="Über der Klause" />
                <tag k="addr:country" v="DE" />
                <tag k="addr:housenumber" v="4a" />
</node>
```

这条街道名称为"Über der Klause"，直译为"经过某路的路"，一般在道路名中使用介词"Über"很少见，我猜测可能是将description误录入了街道名，在非osm地图服务上搜索此街道，却真的存在这样的一条街名。
![Über der Klause](ueberKlause.png)

当然，这并不能完全证实这个名字，如果地图服务商本身用的就是osm的数据的话。这里不作讨论。

在实际项目中，我还查验了其他几个无法分类的街道名，但结果都是事实存在的，可以看出，至少，这个小样本所代表的慕尼黑附近的街道名数据是比较可靠的。

### 使用prepareDB.py

这个脚本主要有两个功能：

1. 提取"node"和"way"的相关属性，组织层级的地址address，坐标pos及相应的键值对 &rarr; shape_element()
2. 以json格式保存到文件，以便导入至数据库 &rarr; process_map()


针对样本数据运行python prepareDB.py后得到:

In [ ]:
# fileutils package is required
!du -sh munich_germany_k10.osm.json && head -n20 munich_germany_k10.osm.json

In [ ]:
# import json into mongo db
# mongo shell is required, tested version v3.2.11
!mongoimport --db=p3 --collection=munichK10 --type=json --drop munich_germany_k10.osm.json

# expected output
# 2017-07-25T12:26:37.089+0200	connected to: localhost
# 2017-07-25T12:26:37.089+0200	dropping: p3.munichK10
# 2017-07-25T12:26:40.088+0200	[###########.............] p3.munichK10	31.5MB/64.5MB (48.8%)
# 2017-07-25T12:26:42.790+0200	[########################] p3.munichK10	64.5MB/64.5MB (100.0%)
# 2017-07-25T12:26:42.790+0200	imported 238481 documents

### 删除冗余数据
在初步浏览数据后，发现有一部分数据的经纬度都为[0,0]:

In [ ]:
> db.munichK10.find({'pos':[0,0]}).count()

# 39431

考虑到我们下载的是慕尼黑周边的区域，坐标不可能是[0,0]，遂从数据集中删除这个数据项：

In [ ]:
> db.munichK10.update(
    {'pos':[0,0]},
    {$unset:{pos:""}},
    {multi:true})

# WriteResult({ "nMatched" : 39431, "nUpserted" : 0, "nModified" : 39430 })

# 数据概览

In [ ]:
# within Mongo shell
# remove suffix '10' to explore the complete collection
> use p3
> m10=p3.munichK10
> m10.findOne()

# {
# 	"_id" : ObjectId("59771d9fd3887b94873fa162"),
# 	"pos" : [
# 		48.1963217,
# 		11.3599471
# 	],
# 	"type" : "node",
# 	"id" : "128218",
# 	"created" : {
# 		"changeset" : "13511613",
# 		"user" : "burt13de",
# 		"version" : "6",
# 		"uid" : "247670",
# 		"timestamp" : "2012-10-15T20:53:25Z"
# 	}
# }

In [ ]:
# number of documents
> m10.find().count()

# 24841

In [ ]:
# number of nodes
> m10.find({'type':'node'}).count()

# 199050

In [ ]:
# number of ways
> m10.find({'type':'way'}).count()

# 39431

In [ ]:
# unique user
> m10.distinct("created.user").length

# 1935

In [ ]:
# top 3 most contributed users
> m10.aggregate([
    {$group:{'_id':'$created.user', 'count':{$sum:1}}},
    {$sort:{'count':-1}},
    {$limit:3}
])

# { "_id" : "ToniE", "count" : 27018 }
# { "_id" : "BeKri", "count" : 23354 }
# { "_id" : "rolandg", "count" : 18677 }

In [ ]:
# number of users who posted only 1, 2 or 3 times.
> m10.aggregate([
    {$group:{'_id':'$created.user', 'count':{$sum:1}}},
    {$group:{'_id':'$count', user_count:{$sum:1}}},
    {$sort:{'user_count':-1}},
    {$limit:3}
])

# { "_id" : 1, "user_count" : 640 }
# { "_id" : 2, "user_count" : 222 }
# { "_id" : 3, "user_count" : 109 }

In [ ]:
# top 5 amenities
> m10.aggregate([
    {$match:{'amenity':{'$exists':1}}},
    {$group:{'_id':'$amenity', 'count':{$sum:1}}},
    {$sort:{'count':-1}},
    {$limit:5},
])

# { "_id" : "bench", "count" : 552 }
# { "_id" : "parking", "count" : 538 }
# { "_id" : "vending_machine", "count" : 256 }
# { "_id" : "shelter", "count" : 187 }
# { "_id" : "waste_basket", "count" : 180 }

In [ ]:
# top 10 cuisine art of restaurants
> m10.aggregate([
    {$match:{
        'amenity':{$exists:1},
        'amenity':'restaurant',
        'cuisine':{$exists:1}}},
    {$group:{'_id':'$cuisine', 'count':{'$sum':1}}},
    {$sort:{'count':-1}},
    {$limit: 10}
])

# { "_id" : "italian", "count" : 37 }
# { "_id" : "regional", "count" : 11 }
# { "_id" : "asian", "count" : 8 }
# { "_id" : "vietnamese", "count" : 8 }
# { "_id" : "indian", "count" : 8 }
# { "_id" : "thai", "count" : 5 }
# { "_id" : "chinese", "count" : 5 }
# { "_id" : "bavarian", "count" : 4 }
# { "_id" : "greek", "count" : 4 }
# { "_id" : "german", "count" : 4 }

# 探索数据
为了能对字符串进行搜索，对真个数据集的‘text’建立索引：

In [ ]:
> m10.createIndex( {"$**":"text"})

#	"createdCollectionAutomatically" : false,
#	"numIndexesBefore" : 2,
#	"numIndexesAfter" : 2,
#	"note" : "all indexes already exist",
#	"ok" : 1

### 问题一
在使用mongodb时，有一部分查询会给出报错信息：

> uncaught exception: can't have . in field names [fields.footway]

这是因为在prepareDB.py中，我将所有tag的属性k，v值，直接添加到node或way的{k:v}字典中，而原本作为属性k的值是可能包含"."符号的。例如：
```
<tag k="footway:right.sloped_curb.end" v="0.02" />
```
将被转换为：
```
{
    "footway:right.sloped_curb.end": "0.02"
}
```

而mongodb的fieldname不支持"."。一个解决[方法](https://stackoverflow.com/questions/8429318/how-to-use-dot-in-field-name)是将"."符号转换成unicode "\uff0e"。但我对这类tag的信息并不特别感兴趣，这里就省直接略掉：
```python
if '.' not in key:
    node[key] = value 
```







